# Covariate Drift Analysis

In [1]:
import pandas as pd
import nannyml as nml
from sklearn.pipeline import Pipeline
import pickle
import numpy as np


Select model architecture to analyze. This choice only effects the analysis of the prediction and positive class probability distributions.

In [2]:
arch = "LReg" # Choose "XGB" or "LReg"

Load and time-order cohorts

In [3]:
# Load model and data
with open(f"model/best_model_{arch}", "rb") as fp:
    modelDict = pickle.load(fp)
with open("trainCohort", "rb") as fp:
    train = pickle.load(fp)
with open("trainLabels", "rb") as fp:
    train_y = pickle.load(fp)
with open("externalCohort", "rb") as fp:
    test = pickle.load(fp)
with open("traceCohort", "rb") as fp:
    trace = pickle.load(fp)

features = modelDict['features']
preprocessing_steps = modelDict['model'].steps[:-1]

with open("featureDict", "rb") as fp:
    featureDict = pickle.load(fp)

processing_pipe = Pipeline(steps=preprocessing_steps)
processing_pipe.steps.pop(1)
processing_pipe.steps.pop(1)


# Load data of interest (Training and Trace-Validation sets along with the timeline of evaluations)
timeline = pd.read_csv("Data/f.53.tab",
            delimiter="\t",
            index_col="f.eid",
            usecols=['f.eid','f.53.0.0', 'f.53.1.0'])

data_train = train.copy()
columns = train.columns
train_indx, test_indx, trace_indx = train.index, test.index, trace.index

train = pd.DataFrame(processing_pipe.transform(train), index=train_indx, columns=columns[features])
train = train.merge(timeline.rename(columns={'f.53.0.0':'timestamp'})['timestamp'],
            how='left',
            left_index=True,
            right_index=True)
train = train.rename(featureDict, axis=1)

data_test = test.copy()

test = pd.DataFrame(processing_pipe.transform(test), index=test_indx, columns=columns[features])
test = test.merge(timeline.rename(columns={'f.53.0.0':'timestamp'})['timestamp'],
            how='left',
            left_index=True,
            right_index=True)
test = test.rename(featureDict, axis=1)

data_trace = trace.copy()
 
trace = pd.DataFrame(processing_pipe.transform(trace), index=trace_indx, columns=columns[features])
trace = trace.merge(timeline.rename(columns={'f.53.1.0':'timestamp'})['timestamp'],
            how='left',
            left_index=True,
            right_index=True)
trace = trace.rename(featureDict, axis=1)


train['y_pred_proba'] = modelDict['model'].predict_proba(data_train)[:,1]
train['period'] = 'reference'
train['y_pred'] = np.where(train['y_pred_proba'].values > modelDict['threshold'], 1, 0)
train['y_true'] = train_y

test['y_pred_proba'] = modelDict['model'].predict_proba(data_test)[:,1]
test['period'] = 'analysis'
test['y_pred'] = np.where(test['y_pred_proba'].values > modelDict['threshold'], 1, 0)

trace['y_pred_proba'] = modelDict['model'].predict_proba(data_trace)[:,1]
trace['period'] = 'analysis'
trace['y_pred'] = np.where(trace['y_pred_proba'].values > modelDict['threshold'], 1, 0)

cat_list = []
for col in train.drop(['period','y_true'], axis=1).columns:
    if len(train[col].value_counts()) < 10:
        cat_list.append(col)

for col in cat_list:
    train[col] = train[col].astype("category")
    if col == 'y_true':
        continue
    trace[col] = trace[col].astype("category")
    test[col] = test[col].astype("category")

## Multivariate Distribution Comparison

### PCA Reconstruction Method

In [4]:
column_names = list(train.columns.drop(['timestamp','y_pred', 'y_pred_proba', 'y_true', 'period']))
multi_calc = nml.DataReconstructionDriftCalculator(
    column_names=column_names,
    n_components=0.7,
    timestamp_column_name='timestamp',
    chunk_size = 500
)

multi_calc.fit(pd.concat([train,test], axis=0))
results = multi_calc.calculate(data=trace)


In [5]:
avg = results.filter(period='reference').to_df().reconstruction_error.value.mean()
std = results.filter(period='reference').to_df().reconstruction_error.value.std()
print("Reference Set Reconstruction Error:")
print(f"\tMean Recon Err: \t{avg:.3f}")
print(f"\tStd Recon Err: \t\t{std:.3f}")
print(f"\tTolerances Bounds: ({avg - 3*std:.3f}, {avg + 3*std:.3f})")

avg = results.filter(period='analysis').to_df().reconstruction_error.value.mean()
print(f"\nAnalysis Set Reconstruction Error: {avg:.3f}")

Reference Set Reconstruction Error:
	Mean Recon Err: 	1.835
	Std Recon Err: 		0.053
	Tolerances Bounds: (1.676, 1.993)

Analysis Set Reconstruction Error: 1.713


In [6]:
figure = results.plot()
figure.show()

## Univariate Distribution Comparisons

### Categorical Distance Metric: `Jensen-Shannon Distance`

### Continuous Distance Metric: `Wasserstein Distance`

In [7]:
column_names = list(train.columns.drop(['timestamp','y_true', 'period']))

uni_calc = nml.UnivariateDriftCalculator(
    column_names=column_names,
    timestamp_column_name='timestamp',
    continuous_methods=['wasserstein'],
    categorical_methods=['jensen_shannon'],
    chunk_size=500
)

uni_calc.fit(pd.concat([train,test], axis=0))
results = uni_calc.calculate(trace)

Plot univariate analysis batches

In [8]:
figure = results.filter(column_names=results.continuous_column_names, methods=['wasserstein']).plot(kind='drift')
figure.show()

In [9]:
figure = results.filter(column_names=results.categorical_column_names, methods=['jensen_shannon']).plot(kind='drift')
figure.show()

In [10]:
figure = results.filter(column_names=results.continuous_column_names, methods=['wasserstein']).plot(kind='distribution')
figure.show()

In [11]:
figure = results.filter(column_names=results.categorical_column_names, methods=['jensen_shannon']).plot(kind='distribution')
figure.show()